<a href="https://colab.research.google.com/github/bbutka/CEC595/blob/main/8b_Language_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let’s write some python code to use all four of the released models for generating text. That will let us see how the changes in capacity related to the quality of the text produced.

We download the GPT-2 library from OpenAI.

The OpenAI codebase has a list of other libraries that it requires, which is handled by installing requirements.txt. We go to the appropriate file, requirements.txt, and install those libraries.

Then, we download four different pre-trained models OpenAI made available, each roughly double in size from the previous.

In [ ]:
!git clone https://github.com/openai/gpt-2.git
import os
os.chdir("gpt-2")
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings('ignore')
%tensorflow_version 1.x
!pip3 install -r requirements.txt
!python3 download_model.py 124M
!python3 download_model.py 345M
!python3 download_model.py 774M
!python3 download_model.py 1558M

Cloning into 'gpt-2'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 4.38 MiB | 4.17 MiB/s, done.
Resolving deltas: 100% (124/124), done.
TensorFlow 1.x selected.
     |████████████████████████████████| 92kB 5.0MB/s 
     |████████████████████████████████| 604kB 12.6MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=54c61c00a34b3093f536897ae5fb20117621e70fb3a8e26c7b8e45d00b27fdda
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
  Created wheel for regex: filename=regex-2017.4.5-cp37-cp37m-linux_x86_64.whl size=534399 sha256=6c9f0daae91c0305e864aad9b43c0d90d5fc95aa933facb36a1865538322a2a6
  Stored in directory: /root/.cache/pip/wheels/75

Fetching checkpoint: 1.00kit [00:00, 964kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 2.43Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 997kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:28, 17.3Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 5.28Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 1.46Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 1.61Mit/s]                                                       
Fetching checkpoint: 1.00kit [00:00, 1.11Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 2.29Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.01Mit/s]                                          

Next we import some addtional libraries we'll be using in this notebook.

In [ ]:
!export PYTHONIOENCODING=UTF-8
os.chdir('src')

!pip install tensorflow=='1.15.2'
import model, sample, encoder
import json
import numpy as np
import tensorflow as tf

  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=cd689dd92110f36f007d72094878a040e5f852a9d25506715533a1a099fc4ddc
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


We define an `autocomplete` function that returns the next `length` number of words given the `model_name` and the `raw_text` input.

We set up a session for talking to the tensorflow backend. We also create a place for the output of the model to go. We checkpoint the tensorflow backend so we can establish the link to our code.Once all of that is set up, we can send our text prompt to the model for processing. We pull out the output of the model and return the string.

In [ ]:
# Return-a-string version

def autocomplete(model_name, raw_text, length):
    batch_size = 1
    temperature = 1
    top_k = 0
    models_dir = '../models'
    seed = None
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        context_tokens = enc.encode(raw_text)
        out = sess.run(output, feed_dict={
                context: [context_tokens]
        })[:, len(context_tokens):]
        text = enc.decode(out[0])
    return(text)

Below is an example of our `autocomplete` function, printing out the next 10 predicted words.

In [ ]:
print(autocomplete('124M', "Learning about machine learning is kind of like", 10))





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from ../models/124M/model.ckpt
 learning how to do a tree inspection for obesity,


Here show how the predictions for a given phrase changes with the number of parameters in the model.

In [ ]:
for gpt2model in ['124M', '345M', '774M', '1558M']:
  print(gpt2model, autocomplete(gpt2model, "My first time visiting the ocean, I marveled at", 20))

INFO:tensorflow:Restoring parameters from ../models/124M/model.ckpt
124M  how clear seas are and the small craters of the ocean below. The depth of my sitting with
INFO:tensorflow:Restoring parameters from ../models/345M/model.ckpt
345M  the view, squinting at the pink and gold. The grass is covered in soft, gl
INFO:tensorflow:Restoring parameters from ../models/774M/model.ckpt
774M  the scale of the world. The Ammarani are famous for the length of their height, the
INFO:tensorflow:Restoring parameters from ../models/1558M/model.ckpt
1558M  the pure impulse behind denying obligation to creatures who don't mind doing that work for you. Only humans


In [ ]:
for gpt2model in ['124M', '345M', '774M', '1558M']:
  print(gpt2model, autocomplete(gpt2model, "When I first arrived at college, I could not believe", 20))

INFO:tensorflow:Restoring parameters from ../models/124M/model.ckpt
124M  that I would be part of any of the institutions that Lloyd´s Short craft commissions focus on:
INFO:tensorflow:Restoring parameters from ../models/345M/model.ckpt
345M  that this was going to be my field. It's now here. But he got me where I
INFO:tensorflow:Restoring parameters from ../models/774M/model.ckpt
774M  the real world had reduced me to an analyst. I was doing research on robots and application design thinking
INFO:tensorflow:Restoring parameters from ../models/1558M/model.ckpt
1558M  how much I was still limited to 20 hours a week of classes. Over the course of my final


In [14]:
for gpt2model in ['124M', '345M', '774M', '1558M']:
  print(gpt2model, autocomplete(gpt2model, "The color of broccoli is a deep", 1))

INFO:tensorflow:Restoring parameters from ../models/124M/model.ckpt
124M  red
INFO:tensorflow:Restoring parameters from ../models/345M/model.ckpt
345M  golden
INFO:tensorflow:Restoring parameters from ../models/774M/model.ckpt
774M ,
INFO:tensorflow:Restoring parameters from ../models/1558M/model.ckpt
1558M ,
